In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter
import sys
sys.path.extend(['..'])
from utils import Config
from data import LSTMCSVDataset
import copy 
from tqdm import tqdm
import os
import json
from datetime import datetime
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

In [2]:
class GestureCSVDatasetv2(torch.utils.data.Dataset):
    def __init__(self, root_path, output_type='quaternion', look_back=10, step_value = 1):
        files = os.listdir(root_path)
        files = [f for f in files if f.endswith('.csv')]
        self.stats = {
                'up': 0,
                'down': 0,
                'forward': 0,
                'backward': 0,
                'none': 0
        }
        gestures = []
        for file in files:
            csv_data = pd.read_csv(os.path.join(root_path, file))
            up_gestures_idx = csv_data[csv_data['gesture']=='Up0'].index.to_numpy()
            down_gestures_idx = csv_data[csv_data['gesture']=='Down0'].index.to_numpy()
            forward_gestures_idx = csv_data[csv_data['gesture']=='Forward0'].index.to_numpy()
            backward_gestures_idx = csv_data[csv_data['gesture']=='Backward0'].index.to_numpy()
            gestures_idx = np.concatenate([up_gestures_idx, down_gestures_idx, forward_gestures_idx, backward_gestures_idx])
#             print(gestures_idx.shape)
            
            csv_data['relativeHandRPosx'] = csv_data['headPosx']-csv_data['handRPosx']
            csv_data['relativeHandRPosy'] = csv_data['headPosy']-csv_data['handRPosy']
            csv_data['relativeHandRPosz'] = csv_data['headPosz']-csv_data['handRPosz']
            csv_data['relativeHandLPosx'] = csv_data['headPosx']-csv_data['handLPosx']
            csv_data['relativeHandLPosy'] = csv_data['headPosy']-csv_data['handLPosy']
            csv_data['relativeHandLPosz'] = csv_data['headPosz']-csv_data['handLPosz']
#             csv_data['relativeTracker1Posx'] = csv_data['headPosx']-csv_data['tracker1Posx']
#             csv_data['relativeTracker1Posy'] = csv_data['headPosy']-csv_data['tracker1Posy']
#             csv_data['relativeTracker1Posz'] = csv_data['headPosz']-csv_data['tracker1Posz']
            fields = set(csv_data.keys())
            needed_feats = []
            if output_type=='quaternion':
                needed_feats = ['headPosy', 'headRotQx', 'headRotQy', 'headRotQz', 'headRotQw',
                    'relativeHandRPosx', 'relativeHandRPosy', 'relativeHandRPosz', 'handRRotQx', 'handRRotQy', 'handRRotQz', 'handRRotQw',
                    'relativeHandLPosx', 'relativeHandLPosy', 'relativeHandLPosz', 'handLRotQx', 'handLRotQy', 'handLRotQz', 'handLRotQw']
            remove_fields = list(fields - set(needed_feats))
            for field in remove_fields:
                csv_data = csv_data.drop(columns=[field])
#             print(fields)
            data = np.array([csv_data]).squeeze(0)
#             print(data.shape)
            scaler = MinMaxScaler(feature_range=(-1,1))
            scaler.fit(data)
            scaled_data = scaler.transform(data)
            # lstm_data = []
            print()
            for i in range(len(scaled_data)-look_back-1):
                a = scaled_data[i:(i+look_back):step_value]
                if self.check_proximal_gesture_type(i, up_gestures_idx):
                    gestures.append((a,1))
                    self.stats['up']+=1
                elif self.check_proximal_gesture_type(i, down_gestures_idx):
                    gestures.append((a,2))
                    self.stats['down']+=1
                elif self.check_proximal_gesture_type(i, forward_gestures_idx):
                    gestures.append((a,3))
                    self.stats['forward']+=1
                elif self.check_proximal_gesture_type(i, backward_gestures_idx):
                    gestures.append((a,4))
                    self.stats['backward']+=1
                else:
                    gestures.append((a,0))
                    self.stats['none']+=1
                # lstm_data.append(a)
        self.gestures = gestures

    def __len__(self):
        return len(self.gestures)
    
    def __getitem__(self, idx):
        return self.gestures[idx][0], self.gestures[idx][1]
        
    def check_proximal_gesture_type(self, idx, gestures_idx, thresh=10):
        for jdx in gestures_idx:
            if abs(idx-jdx)<thresh:
                return True
        return False

In [3]:
dataset = GestureCSVDatasetv2(os.path.join('..',Config['dataset_path']))

In [4]:
len(dataset)

61627

In [5]:
class LSTMClassifier(nn.Module):
    def __init__(self, input_size, output_size):
        super(LSTMClassifier,self).__init__()
        self.input_size = input_size
        self.output_size = output_size

        self.lstm_1 = nn.LSTM(self.input_size, 100, batch_first=True)
        self.fc1 = nn.Linear(1000, 512)
        self.bn2 = nn.BatchNorm1d(512)
        self.fc2 = nn.Linear(512, 256)
        self.bn3 = nn.BatchNorm1d(256)
        self.fc3 = nn.Linear(256, 64)
        self.bn4 = nn.BatchNorm1d(64)
        self.fc4 = nn.Linear(64, output_size)
        self.h1 = self.init_hidden(batch_size=Config['batch_size'],device='cuda:0')

    def init_hidden(self, batch_size, device='cpu'):
        return (torch.zeros(1, batch_size , 100).to(device), torch.zeros(1, batch_size , 100).to(device))
    
    def forward(self, x, device='cpu'):
        out, self.h1 = self.lstm_1(x, self.h1)
#         print(out.shape)
        out = out.reshape(out.shape[0],-1)
        out = nn.Tanh()(self.bn2(self.fc1(out)))
        out = nn.Tanh()(self.bn3(self.fc2(out)))
        out = nn.Tanh()(self.bn4(self.fc3(out)))
        out = self.fc4(out)
#         print(out.shape)
        return out

In [6]:
def train_lstm(dataloader, dataset_sizes, model, criterion, optimizer, device, num_epochs=Config['num_epochs'], batch_size=Config['batch_size']):
    print(Config)

    start_time = datetime.now()
    print("Start Time =", start_time)

    model.to(device)

    best_loss = 100.0
    best_wts = copy.deepcopy(model.state_dict())
    dummy_input = None

    if Config['tensorboard_log']:
        writer = SummaryWriter(Config['model_path'])
    
    for epoch in range(num_epochs):
        for phase in ['train', 'valid']:
            if phase=='train':
                model.train()
            else:
                model.eval()
            
            running_loss = 0.0
            running_corrects = 0 
            running_total = 0
            print(dataloader[phase])
            for inputs, labels in tqdm(dataloader[phase]):
                print(inputs.shape, labels.shape)
                # if inputs.shape[0]!=Config['batch_size']:
                    # continue
                optimizer.zero_grad()
                with torch.set_grad_enabled(phase=='train'):
                    inputs = inputs.to(device)
                    labels = labels.to(device)
                    print('Loaded')
                    model.h1= model.init_hidden(batch_size=inputs.shape[0], device='cuda:0')

                    if dummy_input is None:
                        dummy_input = inputs
                    print('Forward prop')
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
#                     print(outputs.shape, labels[:,-1,:].shape)
                    loss = criterion(outputs, labels)
                    if Config['use_cuda']:
                        l2_regularization = torch.tensor(0.).cuda()
                    else:
                        l2_regularization = torch.tensor(0.)
                    
                    for param in model.parameters():
                        l2_regularization += torch.norm(param, 2)**2

                    loss += 1e-5 * l2_regularization
                    print('Backprop')
                    if phase =='train':
                        loss.backward()
                        optimizer.step()
                        model.h1[0].detach_()
                    running_loss += loss.item()*inputs.size(0)
                    running_corrects += (preds==labels).sum().item()
                    running_total += inputs.size(0)
            epoch_loss = running_loss/running_total
            epoch_acc = running_corrects/running_total
            print(f'Epoch {epoch} Loss: {epoch_loss:.4f} Accuracy: {epoch_acc:.4f}')
            if epoch_loss < best_loss and phase=='valid':
                best_wts = copy.deepcopy(model.state_dict())
                print(outputs[-1,:], labels[-1,-1,:])
            writer.add_scalar(phase+'_loss', epoch_loss, global_step=epoch)
        if (epoch+1)%5==0:
            model.h1 = model.init_hidden(batch_size=1, device='cuda:0')
            torch.onnx.export(model,
                      dummy_input[0].unsqueeze(0),
                      os.path.join(
                            Config['model_path'],
                            f'checkpoints/model_{epoch}.onnx'
                        ),
                      )
            torch.save(
                model.state_dict(),
                os.path.join(
                    Config['model_path'],
                    f'checkpoints/model_{epoch}.pth'
                )
            )
    print('Training ended')
    end_time = datetime.now()
    print("End Time =", end_time)
    print("Total Time =", end_time-start_time)
    torch.save(
        best_wts,
        os.path.join(
            Config['model_path'],
            'checkpoints/model_final.pth'
        )
    )
    model.load_state_dict(best_wts)
    torch.onnx.export(model,
                      dummy_input,
                      os.path.join(
                            Config['model_path'],
                            'checkpoints/model_final.onnx'
                      ),
                      export_params=True
                      )

In [ ]:

os.makedirs(Config['model_path'], exist_ok=True)
os.makedirs(os.path.join(Config['model_path'],'logs'),exist_ok=True)
os.makedirs(os.path.join(Config['model_path'],'checkpoints'),exist_ok=True)

# dataset = LSTMCSVDataset(root_path=Config['dataset_path'])
model = LSTMClassifier(input_size=19, output_size=5)

device = torch.device('cuda:0' if torch.cuda.is_available() and Config['use_cuda'] else 'cpu')

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3) #rmsprop, adam

criterion = torch.nn.CrossEntropyLoss()

ratio = [int(len(dataset)*0.9), len(dataset)-int(len(dataset)*0.9)]
train_dataset, valid_dataset = torch.utils.data.random_split(dataset, ratio)

train_loader = torch.utils.data.DataLoader(
                    train_dataset,
                    batch_size = Config['batch_size'],
                    shuffle=True,
                    num_workers = Config['num_workers']
                )

valid_loader = torch.utils.data.DataLoader(
                    valid_dataset,
                    batch_size = Config['batch_size'],
                    shuffle=False,
                    num_workers = Config['num_workers']
                )
dataloaders = {
    'train' : train_loader,
    'valid' : valid_loader
}

dataset_sizes = {
    'train' : len(train_dataset),
    'valid' : len(valid_dataset)
}
print(dataset_sizes)
train(
    dataloaders, 
    dataset_sizes,
    model,
    criterion,
    optimizer,
    device
    )

{'train': 55464, 'valid': 6163}
{'dataset_path': 'Data_Exp/Exp1', 'model_path': 'models/lstm_classifier', 'onnx_model_path': 'exp1_lstm_model_final.onnx', 'tensorboard_log': True, 'use_cuda': True, 'lr': 0.001, 'num_epochs': 1000, 'batch_size': 256, 'num_workers': 2, 'data_type': 'relative'}
Start Time = 2021-04-22 18:09:59.464731


  0%|                                                                                                                                       | 0/217 [00:00<?, ?it/s]

In [ ]:
print